In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec

In [ ]:
history_txt1 = Path(
    r"1753634027976959_Search_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.05\sim_history.txt"
).read_text(encoding="utf-8")
history_data1 = np.array(
    [
        *map(
            lambda line: [*map(float, line.split(":"))],
            history_txt1.splitlines(),
        )
    ]
)
# print(history_data1)
(
    n_gas_history1,
    n_crystal_history1,
    concentration_history1,
    delta_gibbs_history1,
    energy_change_history1,
    crystal_sx_history1,
    crystal_sy_history1,
    crystal_sz_history1,
    mk_step_history1,
) = history_data1

In [ ]:
from dataclasses import dataclass
from functools import partial
from typing import Callable, List, Tuple

import numpy as np
from scipy.optimize import curve_fit
from scipy.optimize import minimize as opt_minimize


# def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Compute coefficient of determination."""
#     ss_res = np.sum((y_true - y_pred) ** 2)
#     ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
#     return 1 - ss_res / ss_tot


def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Compute coefficient of determination."""

    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")

    if y_true.size < 2:  # R^2 не определен для менее чем 2 точек
        return np.nan  # Или можно поднять ошибку

    y_mean = np.mean(y_true)

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - y_mean) ** 2)

    # Обработка случая, когда ss_tot равно нулю (все y_true одинаковы)
    if ss_tot == 0:
        return (
            1.0 if ss_res == 0 else 0.0
        )  # Если все точки одинаковы и модель их точно предсказывает, R^2 = 1

    return 1 - (ss_res / ss_tot)


@dataclass
class ModelSpec:
    name: str
    t1: str
    t2: str
    func: Callable
    init_guess: List[float]
    bounds: List


MODELS = {
    0: {
        "name": "linear",
        "t1": "c0 * x + c1",
        "t2": "{0} * x + {1}",
        "func": lambda x, c0, c1: c0 * x + c1,
        "init_guess": [-0.00014382693355996512, 25930.775363018805],
        "bounds": ((-np.inf, -np.inf), (np.inf, np.inf)),
    },
    1: {
        "name": "exp001",
        "t1": "c0 * EXP[c1 * x] + c2",
        "t2": "{0} * EXP[{1} * x] + {2}",
        "func": lambda x, c0, c1, c2: c0 * np.exp(c1 * x) + c2,
        "init_guess": [1186.911509826299, -4.202561431523349e-05, 25824.243543934055],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
    2: {
        "name": "exp002",
        "t1": "c0 * EXP[c1 * x]\n + c2 * EXP[c3 * x]\n + c4",
        "t2": "{0} * EXP[{1} * x]\n + {2} * EXP[{3} * x]\n + {4}",
        "func": lambda x, c0, c1, c2, c3, c4: c0 * np.exp(c1 * x)
        + c2 * np.exp(c3 * x)
        + c4,
        "init_guess": [0.09488065437211735, 6.697591186684924e-06, 1194.627363917152, -4.042924858937313e-05, 25810.560463182235],
        "bounds": (
            (-np.inf, -np.inf, -np.inf, -np.inf, -np.inf),
            (np.inf, np.inf, np.inf, np.inf, np.inf),
        ),
    },
    3: {
        "name": "log001",
        "t1": "c1 * LN[x + c2] + c3",
        "t2": "{0} * LN[x + {1}] + {2}",
        "func": lambda x, c1, c2, c3: c1 * np.log(x + c2) + c3,
        "init_guess": [0.01, 1e3, -0.01],
        "bounds": ((-np.inf, 1e-10, -np.inf), (np.inf, np.inf, np.inf)),
    },
    4: {
        "name": "pow001",
        "t1": "c1 * (x ^ c2) + c3",
        "t2": "{0} * (x ^ {1}) + {2}",
        "func": lambda x, c1, c2, c3: c1 * (x**c2) + c3,
        "init_guess": [1e-17, 0.5, -1e-17],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
}


models = {
    model["name"]: ModelSpec(
        name=model["name"],
        t1=model["t1"],
        t2=model["t2"],
        func=model["func"],
        init_guess=model["init_guess"],
        bounds=model["bounds"],
    )
    for model_id, model in MODELS.items()
    if model_id in [0, 1, 2]
}


def find_best_fits(
    x,
    y,
    methods=("lm", "trf", "dogbox"),
):
    results = []
    for method in methods:
        for spec_name, spec in models.items():
            try:
                popt, _ = curve_fit(
                    spec.func,
                    x,
                    y,
                    p0=spec.init_guess,
                    method=method,
                    maxfev=10_000_000,
                )
                y_pred = spec.func(x, *popt)
                r2 = calculate_r2(y, y_pred)
                results.append((method, spec.name, popt.tolist(), r2))
            except Exception:
                continue

    return results


# func_m = lambda x1, x2: np.sqrt(np.sum(np.power(x1 - x2, 2)))
# func_m = lambda x1, x2: np.sum((x1 - x2) ** 2)
func_m = lambda x1, x2: (1 - calculate_r2(x1, x2)) ** 2

func1_base = lambda popt, x, y, func: func_m(y, func(x, *popt))


# def find_best_fits(x, y):
#     methods = [
#         "Nelder-Mead",
#         "Powell",
#         "CG",
#         "BFGS",
#         "L-BFGS-B",
#         "TNC",
#         "COBYLA",
#         "SLSQP",
#         "trust-constr",
#     ]

#     results = []
#     for method in methods:
#         for spec in models:
#             try:
#                 objective_function = partial(func1_base, x=x, y=y, func=spec.func)

#                 initial_guess = spec.init_guess

#                 bounds = [(None, None) for _ in initial_guess]
#                 # bounds = spec.bounds

#                 result = opt_minimize(
#                     objective_function, initial_guess, bounds=bounds, method=method
#                 )
#                 # print(result)

#                 popt = result.x

#                 y_pred = spec.func(x, *popt)
#                 r2 = calculate_r2(y, y_pred)
#                 results.append((method, spec.name, popt.tolist(), r2))
#             except Exception:
#                 continue

#     return results


if __name__ == "__main__":
    # Example usage
    x_data = np.array(mk_step_history1)  # replace with actual data
    y_data = np.array(n_crystal_history1)

    fits = find_best_fits(x_data, y_data)
    # print(fits)
    fits = sorted(fits, key=lambda tup: tup[3], reverse=True)
    best = {}
    for method, model_name, params, r2 in fits:
        # print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
        if model_name not in best:
            best[model_name] = {"m": models[model_name], "params": params, "r2": r2}
            print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")

In [ ]:
# print(best1["exp001"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best1["exp001"]["params"])))
# print(best1["exp002"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best1["exp002"]["params"])))
# print()
# print(best2["exp001"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best2["exp001"]["params"])))
# print(best2["exp002"]["m"].t2.format(*map(lambda v: f"{v:.5e}", best2["exp002"]["params"])))

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path

# === 1. Глобальный стиль ===
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2.0,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

# === 2. Фигура под журнальную колонку ===
fig, ax = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600)

datasets = [
    ("ΔX", mk_step_history1, crystal_sx_history1, "s", "-", 0.55, ".50"),
    ("ΔY", mk_step_history1, crystal_sy_history1, "o", "-", 0.55, ".25"),
    ("ΔZ", mk_step_history1, crystal_sz_history1, "^", "-", 0.55, ".75"),
]

for label, x, y, marker, ls, lw, color in datasets:
    ax.plot(
        x,
        y,
        marker=marker,
        linewidth=lw,
        linestyle=ls,
        color=color,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=mk_step_history1.size // 10,
        label=label,
    )

ax.set_xlabel("Number of $MCS$")
ax.set_ylabel("Size")
ax.legend(loc="best", frameon=True)

# === 6. Оформление сетки и делений ===
ax.grid(True, which="both", linestyle=":", linewidth=0.4)
ax.tick_params(length=2, width=0.5)

# === 7. Финализация и сохранение ===
plt.tight_layout()

# plt.savefig(
#     Path(  #
#         r"_materials\article files\figs"
#         #
#     )
#     / "fig_9_4.png",
#     format="png",
# )
plt.show()
plt.close()

In [ ]:
print("{:.0e}".format(1000000000))

In [ ]:
[
    66 * 20 * 0.41 * 7.38546e-11 * 1.82977e-10,
    20 * 23 * 0.54 * 1.82977e-10 * 2.42724e-10,
    23 * 66 * 0.22 * 2.42724e-10 * 7.38546e-11,
]

In [ ]:
[
    66 * 19 * 0.41 * 7.38546e-11 * 1.82977e-10,
    19 * 22 * 0.54 * 1.82977e-10 * 2.42724e-10,
    22 * 66 * 0.22 * 2.42724e-10 * 7.38546e-11,
]

In [ ]:
[
    54 * 16 * 0.41 * 7.38546e-11 * 1.82977e-10,
    16 * 19 * 0.54 * 1.82977e-10 * 2.42724e-10,
    19 * 54 * 0.22 * 2.42724e-10 * 7.38546e-11,
]

In [ ]:
Ax = 5.85E-10
Ay = 1.78E-10
Az = 4.41E-10

# A1x = Ax / (2 ** (1 / 3))
# A1y = Ay / (2 ** (1 / 3))
# A1z = Az / (2 ** (1 / 3))

# A1x = Ax / (7 ** (1 / 3))
# A1y = Ay / (7 ** (1 / 3))
# A1z = Az / (7 ** (1 / 3))

A1x = Ax / (14 ** (1 / 3))
A1y = Ay / (14 ** (1 / 3))
A1z = Az / (14 ** (1 / 3))

g100 = 0.41
g010 = 0.54
g001 = 0.22

Ω = A1x * A1y * A1z

EX = 2 * g100 * A1y * A1z
EY = 2 * g010 * A1z * A1x
EZ = 2 * g001 * A1x * A1y

Eisol = EX + EY + EZ
E3 = EY + EX
E2 = EZ + EY
E1 = EZ + EX

kB = 1.380649e-23
T = 300.0
kT = kB * T
Ceq = np.exp(- Eisol / kT)

print(
    """
Sizes:
    Ax: {:.5e},    Ay: {:.5e},    Az: {:.5e}
   A1x: {:.5e},   A1y: {:.5e},   A1z: {:.5e}

Volume:
   Ω = A1x * A1y * A1z = {:.5e} * {:.5e} * {:.5e} = {:.5e}

Surface tensions:
   g100: {:.5e}, g010: {:.5e}, g001: {:.5e}

Energys:
1)
   EX = 2.0 * g100 * A1y * A1z = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EY = 2.0 * g010 * A1z * A1x = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}
   EZ = 2.0 * g001 * A1x * A1y = 2.0 * {:.5e} * {:.5e} * {:.5e} = {:.5e}

2)
   Eisol = EX + EY + EZ = {:.5e} + {:.5e} + {:.5e} = {:.5e}

3)
   E3 = EY + EX = {:.5e} + {:.5e} = {:.5e}
   E2 = EZ + EY = {:.5e} + {:.5e} = {:.5e}
   E1 = EZ + EX = {:.5e} + {:.5e} = {:.5e}

kB = {:.5e}; T = {:.5f};
kT = kB * T = {:.5e} * {:.5f} = {:.5e};
Ceq = exp(- Eisol / kT) = exp(- {:.5e} / {:.5e} ) = {:.5e};
""".format(
        Ax, Ay, Az,
        A1x, A1y, A1z,
        A1x, A1y, A1z,
        Ω,
        g100, g010, g001,
        g100, A1y, A1z, EX,
        g010, A1z, A1x, EY,
        g001, A1x, A1y, EZ,
        EX, EY, EZ, Eisol,
        EY, EX, E3,
        EZ, EY, E2,
        EZ, EX, E1,
        kB, T,
        kB, T, kT,
        Eisol, kT, Ceq,
    ).strip()
)

In [ ]:
print(
    "{:.5e}".format(
        np.exp(
            4
            * ((g001 * g010 * g100) ** (1 / 3))
            * (Ω ** (2 / 3))
            / (kT * (27000 ** (1 / 3)))
        )
    )
)
print(
    "{:.5e}".format(
        np.exp(-(Eisol / kT))
        * np.exp(
            4
            * ((g001 * g010 * g100) ** (1 / 3))
            * (Ω ** (2 / 3))
            / (kT * (27000 ** (1 / 3)))
        )
    )
)
print(
    "g010 / g100: {:.5f},\ng001 / g100: {:.5f},\ng010 / g001: {:.5f}".format(
        g010 / g100, g001 / g100, g010 / g001
    )
)
print(
    "∆Y⋅a1y∆X⋅a1x: {:.5f},\n∆Z⋅a1z∆X⋅a1x: {:.5f},\n∆Y⋅a1y∆Z⋅a1z: {:.5f}".format(
        (66 * A1y) / (23 * A1x), (20 * A1z) / (23 * A1x), (66 * A1y) / (20 * A1z)
    )
)

In [ ]:
((g010 / g100)), ((66 * A1y) / (23 * A1x)), ((g010 / g100)) / ((66 * A1y) / (23 * A1x))

In [ ]:
((g001 / g100)), ((20 * A1z) / (23 * A1x)), ((g001 / g100)) / ((20 * A1z) / (23 * A1x))

In [ ]:
((g010 / g001)), ((66 * A1y) / (20 * A1z)), ((g010 / g001)) / ((66 * A1y) / (20 * A1z))